# GUARDSHIELD_CNNLSTM 
# Version 1.0

## Loading Dataset

In [1]:
import pandas as pd

df = pd.read_csv("hf://datasets/racdroid/cicds-2017/cleaned_data.csv", engine='pyarrow')

c:\Users\laksh\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset Enumeration

In [2]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,40,0,5.000000e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,1,1,0,9.0,6.0,0.0,40,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,1,1,0,9.0,6.0,0.0,40,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,1,1,0,9.0,6.0,0.0,40,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,1,1,0,9.0,6.0,0.0,40,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,40,0,6.666667e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,1,1,0,9.0,6.0,0.0,40,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [3]:
df.shape

(2827876, 67)

In [4]:
df.dtypes

 Destination Port                int64
 Flow Duration                   int64
 Total Fwd Packets               int64
 Total Backward Packets          int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
 Idle Std                      float64
 Idle Max                        int64
 Idle Min                        int64
 Label                          object
Length: 67, dtype: object

In [5]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06,2.827876e+06
mean,8.061534e+03,1.480065e+07,9.368972e+00,1.040396e+01,5.498522e+02,1.617903e+04,2.078044e+02,1.872929e+01,5.825628e+01,6.897811e+01,8.717304e+02,4.108922e+01,3.062575e+02,3.356657e+02,1.491719e+06,7.085423e+04,1.299765e+06,2.922231e+06,9.191785e+06,1.625442e+05,1.449765e+07,2.612839e+06,3.270269e+06,9.052107e+06,1.022929e+06,9.903861e+06,1.807615e+06,1.487480e+06,4.689442e+06,9.682420e+05,4.639595e-02,-2.602379e+04,-2.275588e+03,6.393010e+04,7.002284e+03,1.644888e+01,9.513591e+02,1.721137e+02,2.952720e+02,4.866468e+05,3.528266e-02,4.639595e-02,2.982836e-01,3.153147e-01,9.482948e-02,6.838443e-01,1.921707e+02,5.825628e+01,3.062575e+02,-2.602379e+04,9.368972e+00,5.498417e+02,1.040396e+01,1.617869e+04,6.992389e+03,1.988290e+03,5.423519e+00,-2.744494e+03,8.163400e+04,4.117582e+04,1.533378e+05,5.835492e+04,8.324468e+06,5.043548e+05,8.704568e+06,7.928061e+06
std,1.827432e+04,3.366750e+07,7.500527e+02,9.978937e+02,9.998639e+03,2.264235e+06,7.175183e+02,6.035533e+01,1.861733e+02,2.813212e+02,1.947157e+03,6.888118e+01,6.054855e+02,8.400508e+02,2.594016e+07,2.544154e+05,4.510039e+06,8.049410e+06,2.447019e+07,2.951772e+06,3.358966e+07,9.530187e+06,9.643379e+06,2.453990e+07,8.595729e+06,2.874945e+07,8.891515e+06,6.281472e+06,1.716900e+07,8.313137e+06,2.103411e-01,2.106353e+07,1.452945e+06,2.476542e+05,3.817038e+04,2.524615e+01,2.029034e+03,3.055985e+02,6.320510e+02,1.648252e+06,1.844934e-01,2.103411e-01,4.575047e-01,4.646412e-01,2.929793e-01,6.805824e-01,3.319734e+02,1.861733e+02,6.054855e+02,2.106353e+07,7.500527e+02,9.985113e+03,9.978937e+02,2.264204e+06,1.434022e+04,8.454537e+03,6.367482e+02,1.085539e+06,6.489234e+05,3.935787e+05,1.026333e+06,5.773818e+05,2.364057e+07,4.605289e+06,2.437766e+07,2.337390e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.610000e+08,-2.000000e+06,-1.300000e+01,0.000000e+00,-1.300000e+01,-1.400000e+01,0.000000e+00,0.000000e+00,0.000000e

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827876 entries, 0 to 2827875
Data columns (total 67 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [7]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count',
       ' SYN Flag Count', ' PSH Flag Count',

## Data Preprocessing

In [8]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Filling missing numerical values with median
df.dropna()


# Encoding labels using LabelEncoder for classification
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df[' Label'])

# Dropping irrelevant columns
df.drop(columns=[' Label'], inplace=True)


# Seperating features and target
X = df.drop(columns=['Label']).values
Y = df['Label'].values


# Normalize feature set using StandardScaler
scaler=StandardScaler()
X=scaler.fit_transform(X)

# Preparing sequences for LSTM
def create_sequences(X, Y, time_steps=5):
    Xs, Ys = [],[]
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        Ys.append(Y[i+time_steps])
    return np.array(Xs), np.array(Ys)

time_steps = 5
X_seq, Y_seq = create_sequences(X,Y, time_steps)

# Splitting data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_seq, Y_seq, test_size=0.2, random_state=42)

# Printing
print(f'X_train_shape: {X_train.shape}, Y_train_shape: {Y_train.shape}, X_test_shape:{X_test.shape}, Y_test_shape: {Y_test.shape}')

X_train_shape: (2262296, 5, 66), Y_train_shape: (2262296,), X_test_shape:(565575, 5, 66), Y_test_shape: (565575,)


## Model Building

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def custom_binary_crossentropy(y_true, y_pred):
    # Clip predictions to prevent log(0) issue
    epsilon = 1e-7
    y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    # Compute binary cross-entropy
    return -tf.reduce_mean(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))

Y_train=label_encoder.fit_transform(Y_train)
Y_test=label_encoder.fit_transform(Y_test)

model=Sequential() 

# CNN Layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# LSTM Layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

history=model.fit(X_train,Y_train, epochs=20, batch_size=64, validation_split=0.2)

loss,accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss : {loss}, Test Accuracy: {accuracy}')

Epoch 1/20
28279/28279 [==============================] - 222s 8ms/step - loss: -86.7401 - accuracy: 0.7141 - val_loss: -166.6111 - val_accuracy: 0.7690
Epoch 2/20
28279/28279 [==============================] - 107s 4ms/step - loss: -243.8585 - accuracy: 0.7319 - val_loss: -323.9262 - val_accuracy: 0.7746
Epoch 3/20
28279/28279 [==============================] - 112s 4ms/step - loss: -400.9970 - accuracy: 0.7189 - val_loss: -481.9239 - val_accuracy: 0.7750
Epoch 4/20
28279/28279 [==============================] - 114s 4ms/step - loss: -558.1447 - accuracy: 0.7106 - val_loss: -639.8616 - val_accuracy: 0.7742
Epoch 5/20
28279/28279 [==============================] - 121s 4ms/step - loss: -715.9796 - accuracy: 0.7057 - val_loss: -798.2824 - val_accuracy: 0.7711
Epoch 6/20
28279/28279 [==============================] - 117s 4ms/step - loss: -873.6999 - accuracy: 0.7018 - val_loss: -956.7368 - val_accuracy: 0.7745
Epoch 7/20
28279/28279 [==============================] - 118s 4ms/step - los